### A4.3.2. Memory Planning

$$
\text{Peak Memory} = \max_{t} \sum_{b \in \text{live}(t)} \text{size}(b)
$$

where $\text{live}(t)$ is the set of buffers alive at time step $t$ in the execution schedule.

**Explanation:**

**Memory planning** (buffer assignment) determines when and where to allocate/deallocate tensor buffers during compiled model execution. The goal is to minimize peak memory usage while respecting data dependencies.

**Key Concepts:**

- **Liveness analysis** — a buffer is live from its definition until its last use. Non-overlapping lifetimes can share the same memory.
- **Buffer aliasing** — reuse the input buffer for the output when the op is in-place safe (e.g., elementwise ops).
- **Memory fragmentation** — even if total free memory suffices, fragmented free regions may prevent allocation.

**XLA Buffer Assignment:**

1. Schedule HLO instructions (topological order with heuristics).
2. Compute live ranges for every buffer.
3. Assign non-overlapping buffers to the same memory offset (coloring problem).
4. Handle constraints: pinned buffers (inputs/outputs), alignment, aliasing hints.

**Strategies:**

| Strategy | Description |
|----------|------------|
| Greedy by size | Assign largest buffers first, reuse biggest available gap |
| Offset assignment | Pack buffers at offsets in a contiguous arena |
| Rematerialization | Recompute a value instead of storing it (trade compute for memory) |
| Gradient checkpointing | Store only selected activations during forward; recompute the rest during backward |

**Example:**

Three buffers A, B, C where A is live during steps 0–2, B during 1–3, C during 3–5. Buffer A and C have non-overlapping lifetimes → they share the same memory slot.

In [ ]:
from dataclasses import dataclass


@dataclass
class Buffer:
    name: str
    size_bytes: int
    live_start: int
    live_end: int


def compute_peak_memory(buffers, total_steps):
    peak = 0
    peak_step = 0
    memory_timeline = []
    for step in range(total_steps):
        live_buffers = [
            buf for buf in buffers
            if buf.live_start <= step <= buf.live_end
        ]
        step_memory = sum(buf.size_bytes for buf in live_buffers)
        memory_timeline.append((step, step_memory, [buf.name for buf in live_buffers]))
        if step_memory > peak:
            peak = step_memory
            peak_step = step
    return peak, peak_step, memory_timeline


def find_sharing_opportunities(buffers):
    opportunities = []
    for index_a in range(len(buffers)):
        for index_b in range(index_a + 1, len(buffers)):
            buf_a = buffers[index_a]
            buf_b = buffers[index_b]
            no_overlap = buf_a.live_end < buf_b.live_start or buf_b.live_end < buf_a.live_start
            if no_overlap:
                opportunities.append((buf_a.name, buf_b.name))
    return opportunities


buffers = [
    Buffer("input", 128 * 784 * 4, 0, 1),
    Buffer("weights", 784 * 256 * 4, 0, 5),
    Buffer("matmul_out", 128 * 256 * 4, 1, 2),
    Buffer("bias_out", 128 * 256 * 4, 2, 3),
    Buffer("relu_out", 128 * 256 * 4, 3, 5),
    Buffer("grad_relu", 128 * 256 * 4, 4, 5),
]

peak, peak_step, timeline = compute_peak_memory(buffers, total_steps=6)

print("Memory timeline:")
for step, memory, live_names in timeline:
    bar = "█" * (memory // (128 * 256))
    print(f"  t={step}: {memory / (1024**2):6.2f} MB  [{', '.join(live_names)}] {bar}")

print(f"\nPeak memory: {peak / (1024**2):.2f} MB at step {peak_step}")

sharing = find_sharing_opportunities(buffers)
print(f"\nBuffer sharing opportunities (non-overlapping lifetimes):")
for buf_a, buf_b in sharing:
    print(f"  {buf_a} ↔ {buf_b}")

**References:**

[📘 OpenXLA Project. *XLA Buffer Assignment.*](https://openxla.org/xla/architecture)

[📘 Chen, T. et al. (2016). *Training Deep Nets with Sublinear Memory Cost.* arXiv:1604.06174.](https://arxiv.org/abs/1604.06174)

---

[⬅️ Previous: Operator Fusion](./01_operator_fusion.ipynb) | [Next: Dispatch ➡️](./03_dispatch.ipynb)